<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Python Kafka Avro Producer 

**Technical Accomplishments:**
- Start working with avro schema in Kafka
- Introduce the class `AvroProducer`
- Produce data for a Kafka avro topic

## Getting Started

Let's start importing libraries and creating useful variables 

In [ ]:
%load_ext autotime

In [4]:
from confluent_kafka import avro
from confluent_kafka.avro import AvroProducer
import time
from confluent_kafka.admin import AdminClient, NewTopic


#import qcutils

servers="broker:29092"

sr_url="http://schema-registry:8081"

topic = 'Avro-simple1'

admin_client = AdminClient({
    "bootstrap.servers": servers
})

topic_list = []
topic_list.append(NewTopic(topic, 1, 1))
admin_client.create_topics(topic_list)

producerconf = {
        'bootstrap.servers': servers,
        'schema.registry.url': sr_url
    }

**Note**: in order to avoid conflicts during write operation, please name the topic as `<surname>-topic`

## Getting Started

Let's start importing libraries and creating useful variables 

## Avro Producer

In [5]:
key_schema_str = """
{
  "namespace": "example.avro",
  "type": "record",
  "name": "PersonKey",
  "fields": [
    {
      "name": "name",
      "type": "string"
    }
  ]
}
"""

value_schema_str = """
{
  "namespace": "example.avro",
  "type": "record",
  "name": "PersonValue",
  "fields": 
    [{
      "name": "age",
      "type": "int",
      "default": 18
    }]
}
"""

key_schema = avro.loads(key_schema_str)
value_schema = avro.loads(value_schema_str)



In [6]:
key = {"name": "Abe"}

def cb(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

#sr_url=qcutils.read_config_value("kafka.schema_registry.url")

producerconf = {
        'bootstrap.servers': servers,
        'schema.registry.url': sr_url
    }
        
ap = AvroProducer(producerconf, default_key_schema=key_schema, default_value_schema=value_schema)

for i in range(0, 100):
    value = {"age": i}
    ap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema, callback=cb)
    print(value)
    ap.poll(0)
    time.sleep(1)
    
ap.flush()

ConnectionError: HTTPConnectionPool(host='schema-registry', port=8081): Max retries exceeded with url: /subjects/Avro-simple1-value/versions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc31c701820>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

## Avro Producer with Evolving Schema

In [ ]:
from confluent_kafka import avro
from confluent_kafka.avro import AvroProducer
import time

key_schema_str = """
{
  "namespace": "example.avro",
  "type": "record",
  "name": "PersonKey",
  "fields": [
    {
      "name": "name",
      "type": "string"
    }
  ]
}
"""

value_schema_str = """
{
  "namespace": "example.avro",
  "type": "record",
  "name": "PersonValue",
  "fields": [
    {
      "name": "name",
      "type": "string"
    },
    {
      "name": "age",
      "type": "int",
      "default": 18
    },
    {
      "name": "haircolor",
      "type": "string",
      "default": "black"
    }
  ]
}
"""

key_schema = avro.loads(key_schema_str)
value_schema = avro.loads(value_schema_str)

key = {"name": "Abe"}

def cb(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

producerconf = {
        'bootstrap.servers': servers,
        'schema.registry.url': sr_url
    }
        
ap = AvroProducer(producerconf, default_key_schema=key_schema, default_value_schema=value_schema)

for i in range(0, 100):
    if i < 40:
        hairColor = "black"
    elif (i >=40 and i < 55):
        hairColor = "grizzled"
    else:
        hairColor = "white"
  
    value = {"name": "Abe", "age": i, "haircolor": hairColor}
    ap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema, callback=cb)
    print(value)
    ap.poll(0)
    time.sleep(1)

ap.flush()

**Note** In order to add SASL security for the connection to the kafka broker, you need to add security configurations.

```
username=qcutils.read_config_value("kafka.access.key")
password=qcutils.read_config_value("kafka.access.secret")

sr_user_info=qcutils.read_config_value("kafka.schema_registry.key") + ":" + qcutils.read_config_value("kafka.schema_registry.secret")

producerconf = {
        'bootstrap.servers': <servers>,
        'sasl.mechanisms': 'PLAIN',
        'security.protocol': 'SASL_SSL',
        'sasl.username': <username>,
        'sasl.password': <password>,
        'schema.registry.url': sr_url,
        'schema.registry.basic.auth.credentials.source': 'USER_INFO',
        'schema.registry.basic.auth.user.info': <login-info>
    }
    
```

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2020 Quantia Consulting, srl. All rights reserved.